In [1]:
import pandas as pd
import numpy as np
# import sklearn as sk
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neural_network import MLPClassifier
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, ActivityRegularization, Add
# from keras.callbacks import ModelCheckpoint
# from keras.wrappers.scikit_learn import KerasClassifier

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,523,7.2500,146,1.0,11
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,596,71.2833,81,2.0,12
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,669,7.9250,146,1.0,8
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,49,53.1000,55,1.0,12
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,472,8.0500,146,1.0,11


In [4]:
# PID, Pclass, Survived, Age, SibSp, Parch, Fare are ints, so will have to convert the rest of the values into ints.
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title']

In [5]:
# Filling the NaN values with the mean of the other values in the particluar column.
for i in features:
    train[i] = train[i].fillna(train[i].mean())
    test[i] = test[i].fillna(test[i].mean())

In [6]:
# Normalising the data
for i in features:
    train[i] = preprocessing.scale(train[i])
    test[i] = preprocessing.scale(test[i])

In [7]:
# Fitting the model on a part of the training set (G).
train, d = train_test_split(train)

In [8]:
# Model

# clf = xgb.XGBClassifier(objective='binary:logistic',learning_rate=0.1)

clf = RandomForestClassifier(criterion='entropy', n_estimators=500,max_depth=10,min_samples_split=4,min_samples_leaf=5, max_features='auto',oob_score=True,n_jobs=-1,verbose=0)

# clf = RandomForestClassifier()

# clf = xgb.XGBClassifier(max_depth=2, n_estimators=1000, learning_rate=0.1, reg_lambda=0.7, reg_alpha=0.4)

# clf = DecisionTreeClassifier(max_depth=11, min_samples_leaf=8)

# clf = MLPClassifier(activation='tanh', solver='adam', alpha=0.1, hidden_layer_sizes=(200, 50), random_state=1, batch_size=32)

In [9]:
clf.fit(train[features], train['Survived'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [10]:
clf.score(train[features], train['Survived'])

0.8832335329341318

In [11]:
clf.score(d[features], d['Survived'])

0.8385650224215246

In [12]:
predictions = clf.predict(test[features])
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predictions})
filename = 'Titanic_Predictions.csv'
submission.to_csv(filename,index=False)

In [13]:
len(submission[submission.Survived ==1 ])

159